## Campaign_Investments_Vs_Provider_Networks

### Table of Content
1. Introduction
2. Data Preprocessing
3. Feature Engineering
4. Exploratory Data Analysis & Statistical Analysis
5. Recommendations & Strategies

### 1. Introduction

In [ ]:
%%capture
!pip install pytimetk -q

In [ ]:
import pandas as pd, numpy as np, seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive/')
import pytimetk as tk
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 2. Data Preprocessing

In [ ]:
data=pd.read_csv(r'/content/drive/MyDrive/campaign_data.csv')
data.columns

### 3. Feature Engineering

In [ ]:
## Time Series Features
data['Calendardate']=pd.to_datetime(data['Calendardate'])
data['Calendardate']=data['Calendardate'].astype('datetime64[ns]')
data['Calendar_Month']=data['Calendardate'].dt.month
data['Calendar_Year']=data['Calendardate'].dt.year

### 4. Exploratory Data Analysis and Statistical Analysis
We can have a detailed exploration that can be added to this section, but since we only need to answer three questions:

### 4.1 Exploratory Data Analysis & Statistical Analysis

#### 4.1.1 Distribution of Features over Time


In [ ]:
value_vars = ['Amount Collected', 'Unit Sold', 'Campaign (Email)',
              'Campaign (Flyer)', 'Campaign (Phone)', 'Sales Contact 1',
              'Sales Contact 2', 'Sales Contact 3']
sns.set_style("darkgrid", {"grid.color": ".5", "grid.linestyle": "--"})

num_vars = len(value_vars)
num_cols = 2
num_rows = (num_vars + num_cols - 1) // num_cols

# Create the figure and subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, num_rows * 6))

# Iterate through the variables and create subplots
for i, var in enumerate(value_vars):
    # Melt the data for the current variable
    data_melted = data.melt(id_vars=["Calendardate"],
                            value_vars=[var],
                            var_name='variable',
                            value_name='value')
    data_melted = data_melted.groupby(['Calendardate', 'variable']).agg(
        {'value': 'sum'}).sort_values(
            by='value', ascending=True).reset_index()
    data_melted = data_melted.set_index('Calendardate')
    data_melted = data_melted.resample('M').sum().fillna(0)
    data_melted = data_melted.reset_index()

    # Get the subplot axes
    row = i // num_cols
    col = i % num_cols
    ax = axes[row, col] if num_rows > 1 else axes[col]

    # Plot the time series on the subplot with smoothed lines
    sns.lineplot(x='Calendardate',
                 y='value',
                 data=data_melted,
                 ax=ax,
                 color='blue',  # Customize color if needed
                 linewidth=2,
                 estimator='sum',  # For smoothing
                 ci=None,
                 label=var)  # Remove confidence interval if not needed

    ax.set_title(var, fontsize=14)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Value', fontsize=12)
    ax.legend()

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

#### 4.1.2 Distribution of Amount Collected

In [ ]:
# Set a more appealing color palette
sns.set_palette("muted")

# Create the histogram with KDE
plt.figure(figsize=(12, 6))
sns.histplot(data['Amount Collected'], bins=30, kde=True, color='blue', line_kws={'color': 'orange'},linewidth=1.5)

# Add mean and median lines for context
mean_value = data['Amount Collected'].mean()
median_value = data['Amount Collected'].median()

plt.axvline(mean_value, color='red', linestyle='-.', label=f'Mean: {mean_value:.2f}')
plt.axvline(median_value, color='green', linestyle='--', label=f'Median: {median_value:.2f}')

# Labels and Title
plt.xlabel("Amount Collected", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Distribution of Amount Collected Across Accounts", fontsize=14)

# Add a legend to explain the lines
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Observations: The distribution is right skewed

#### 4.1.3 Sales Across Account Type

In [ ]:
# Mean Sales across Client Type
sales_mean = data.groupby('Client Type')['Amount Collected'].mean().reset_index()

plt.figure(figsize=(10, 5))
sns.barplot(x='Client Type', y='Amount Collected', data=sales_mean, palette='viridis')
plt.xlabel("Client Type")
plt.ylabel("Mean Amount Collected")
plt.title("Mean Sales Across Account Types")
plt.xticks(rotation=45)
plt.show()
print(sales_mean)

# Total Sales across Client Type
sales_sum = data.groupby('Client Type')['Amount Collected'].sum().reset_index()

plt.figure(figsize=(10, 5))
sns.barplot(x='Client Type', y='Amount Collected', data=sales_sum, palette='viridis')
plt.xlabel("Client Type")
plt.ylabel("Total Amount Collected")
plt.title("Total Sales Across Account Types")
plt.xticks(rotation=45)
plt.show()
print(sales_sum)

Observation:  
*   Large Facilities: While their average sale might be LOWER, they likely have a much LARGER VOLUME OF TRANSACTIONS. This higher volume compensates for the lower average, resulting in a higher total revenue (sum).
*   Medium Facilities: They have the highest average sale, but the total number of transactions might be  compared to Large Facilities. This leads to a lower overall revenue (sum) despite the higher average.

### 4.2 Statistical Analysis - Answering the Questions


#### 4.2.1 Impact of Marketing Strategy on Sales

In [ ]:
data['Client Type'].value_counts(normalize=True).reset_index()

Observations: The Client share is highest for Large facilities followed by Small and Medium

In [ ]:
pd.crosstab(data['Number of Competition'],data['Client Type'],margins=True,normalize='columns')

Observations: We observe that majority of our market is from LOW

In [ ]:
data[['Amount Collected','Unit Sold','Number of Competition']].groupby('Number of Competition').mean().reset_index()

Observations: Even though majority of our market is from LOW, the AVG sales and Units Sold is 2X in HIGH competition

In [ ]:
data[['Number of Customers','Amount Collected','Unit Sold','Client Type']].groupby('Client Type').mean().reset_index()

Observation:
*    Even though "Medium Facilities" does not see much client share, we observe that "Medium Facilities" bring in More Customers, Sales and Units sold.
*   If we see and apply similar marketing strategies to other client groups, then we might see better sales and popultaion

#### 4.2.2 Correlation Analysis

#### (i) Impact of Specific Marketing Strategies on Sales

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
correlation_analysis=pd.DataFrame(data[['Amount Collected',
'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected']).reset_index()
correlation_analysis.columns=['Impacting Variable','Degree of Linear Impact (Correlation)']
correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable'] != 'Amount Collected']
correlation_analysis = correlation_analysis.sort_values('Degree of Linear Impact (Correlation)', ascending=False)
# Apply background gradient and set decimal format correctly
correlation_analysis.style.background_gradient(cmap=cm).format(precision=2)

#### (ii) Impact of Specific Marketing Strategies on Sales across Client Types

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
correlation_analysis=pd.DataFrame(data.groupby('Client Type')[['Amount Collected',
       'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected']).reset_index()
correlation_analysis=correlation_analysis.sort_values(['Client Type','Amount Collected'],ascending=False)
correlation_analysis.columns=['Acc Type','Variable Impact on Sales','Impact']
correlation_analysis=correlation_analysis[correlation_analysis['Variable Impact on Sales']!='Amount Collected'].reset_index(drop=True)
correlation_analysis.style.background_gradient(cmap=cm).format(precision=2)

#### 4.2.3 Causal Inference - Regression Analysis (Revenue Vs Strategies)

In [ ]:
## column clean
data.columns=[mystring.replace(" ", "_") for mystring in data.columns]
data.columns=[mystring.replace("(", "") for mystring in data.columns]
data.columns=[mystring.replace(")", "") for mystring in data.columns]

In [ ]:

results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5',data=data).fit()
print(results.summary())

Observation:

1. We see expected change in the Target (amount collected) with unit change in the variables.
2. However, we need to understand wh of these variables are statistically significant (i.e P<0.05) , implies with 95% confindence we can say that a variable is significant and by analysing the coeff we say to what extent



In [ ]:
df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]
df=df.reset_index()
df=df[df['P>|t|']<0.05][['index','coef','P>|t|']] ## Filter Dataframe for only significant predictors (p<0.05)
df

#### Causal Inference - Regression Analysis (Revenue Vs Strategies across Client Types)

In [ ]:
consolidated_summary=pd.DataFrame()
for acctype in list(set(list(data['Client_Type']))):
    temp_data=data[data['Client_Type']==acctype].copy()
    results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5', data=temp_data).fit()
    df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
    df=df[df['P>|t|']<0.05][['index','coef']]
    df.columns=['Variable','Coefficent (Impact)']
    df['Account Type']=acctype
    df=df.sort_values('Coefficent (Impact)',ascending=False)
    df=df[df['Variable']!='Intercept']
    ##print(acctype)
    consolidated_summary=pd.concat([consolidated_summary, df], ignore_index=True)
    print(df)

In [ ]:

consolidated_summary=pd.DataFrame()
for acctype in list(set(list(data['Client_Type']))):
    print(acctype)
    temp_data=data[data['Client_Type']==acctype].copy()
    results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5', data=temp_data).fit()
    df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
    df=df[df['P>|t|']<0.05][['index','coef']]
    df.columns=['Variable','Coefficent (Impact)']
    df['Account Type']=acctype
    df=df.sort_values('Coefficent (Impact)',ascending=False)
    df=df[df['Variable']!='Intercept']
    consolidated_summary=pd.concat([consolidated_summary, df], ignore_index=True)
    print(results.summary())
    print("***************************************************************************************************************************************************************************")
    print("***********************************************************************************************************************************************************************************")

### 5. Recommendations and Strategies

*   The table below allows us to analyze the return on investment for each dollar spent, providing insights into the effectiveness of various campaigns and sales contacts across different account types.
*  For the Medium Facility case:
  1. Flyer Campaigns yield a strong return, generating an average of $4 for every dollar spent.
  2. Sales Contact 2 is the most effective, followed by Sales Contact 1, with Sales Contact 3 showing less impact.
  3. Other strategies are ineffective and can be eliminated to reduce costs.

In [ ]:
consolidated_summary.columns

In [ ]:
consolidated_summary=consolidated_summary.sort_values(by='Account Type', ascending=True)

In [ ]:
consolidated_summary.reset_index(inplace=True)
consolidated_summary.drop('index',inplace=True,axis=1)

In [ ]:
consolidated_summary.columns = ['Variable','Return on Investment','Account Type']
consolidated_summary['Return on Investment']= consolidated_summary['Return on Investment'].apply(lambda x: round(x,1))
consolidated_summary.style.background_gradient(cmap='RdYlGn')

In [ ]:
def format(x):
        return "${:.1f}".format(x)
consolidated_summary['Return on Investment']  = consolidated_summary['Return on Investment'].apply(format)

In [ ]:
consolidated_summary.columns = ['Variable','Return on Investment','Account Type']
consolidated_summary.style.background_gradient(cmap='RdYlGn')

## 5. Recommendations & Strategies

In [ ]:
# 1. Large Facility:
#    - Top Contacts: Prioritize Sales Contact 1 ($11.7) and Sales Contact 4 ($10.6) for maximum ROI.
#    - Flyers ($2.7) provide moderate returns; consider using them as a secondary strategy.
#    - Phone Campaigns (-$3.5) should be discontinued due to negative ROI.

# 2. Medium Facility:
#    - Flyer Campaigns ($4.1) deliver strong returns and should be prioritized.
#    - Sales Contact 2 ($3.6) is the most effective, followed by Sales Contact 1 ($3.1). Sales Contact 3 ($2.1) can be minimized.

# 3. Private Facility:
#    - Sales Contact 2 ($6.6) offers the highest return and should be the primary focus.

# 4. Small Facility:
#    - Sales Contact 2 ($0.8) is yielding minimal returns; consider testing alternatives.
#    - Phone Campaign ($0.0) shows no significant impact and should be reconsidered.

# ### Suggestions:
# - Focus on High-ROI Strategies: Prioritize the best-performing contacts and campaigns for each account type.
# - Eliminate Ineffective Campaigns: Discontinue low-impact strategies like Phone Campaigns in Large and Small Facilities.
# - Allocate Resources Wisely: Invest more in Flyer Campaigns for Medium Facilities and Sales Contact 2 for Private Facilities.
